In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split

### 1. Wczytaj końcowy i przetworzony zbiór danych Titanic z poprzednich zajęć.

In [2]:
df = pd.read_csv('dane.csv')
df.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,CabinReduced
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO",B
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",C
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",C
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,135.0,"Montreal, PQ / Chesterville, ON",C
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",C
5,1.0,1,"Anderson, Mr. Harry",male,48.0000,0.0,0.0,19952,26.5500,E12,S,3,NaN,"New York, NY",E
6,1.0,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1.0,0.0,13502,77.9583,D7,S,10,NaN,"Hudson, NY",D
7,1.0,0,"Andrews, Mr. Thomas Jr",male,39.0000,0.0,0.0,112050,0.0000,A36,S,NaN,NaN,"Belfast, NI",A
8,1.0,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2.0,0.0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY",C
9,1.0,0,"Artagaveytia, Mr. Ramon",male,71.0000,0.0,0.0,PC 17609,49.5042,NaN,C,NaN,22.0,"Montevideo, Uruguay",N


### 2. Zapoznaj się z funkcja train_test_split wchodzącą w skład biblioteki Scikit-learn. Zapisz swoje obserwacje.
- Funckja train_test_split - dzieli w sposób przypadkowy ramkę danych na zbiór treningowy i testowy
- Podajemy procent jaki chcemy uzyskać w poszczególnych zbiorach, aczkolwiek funckja defaultowa rozbija zbiór na 75% train i 25% test
- random_state - to ziarno, którym ustalamy sposób randomowości, także by zapewnić reprodukowalność wyników
- Wszystko to moglibyśmy zrobić sami, aczkolwiek w ten sposób zapis jest szybki i przejrzysty

### 3. Stwórz zmienną do której zapiszesz listę z nazwami trzech kolumn – kabiny, zredukowane kabiny oraz płeć. Nazwij ją col_name.

In [3]:
col_name = ['cabin','CabinReduced','sex']

### 4. Podziel zbiór na treningowy i testowy używając train_test_split.

In [4]:
X = df[col_name]
y = df['survived']
X_train, X_test, y_train, y_test  = train_test_split(X,y,train_size = 0.7, test_size = 0.3, random_state = 0)
print('X_train shape:', X_train.shape, f',Train size / Df size : {round(len(X_train)/len(df),3)}')
print('X_test shape:',X_test.shape,f',Test size / Df size : {round(len(X_test)/len(df),3)}')
print('y_train shape:',y_train.shape, f',Train size / Df size : {round(len(y_train)/len(df),3)}')
print('y_test shape:',y_test.shape,f',Test size / Df size : {round(len(y_test)/len(df),3)}')

X_train shape: (916, 3) ,Train size / Df size : 0.7
X_test shape: (393, 3) ,Test size / Df size : 0.3
y_train shape: (916,) ,Train size / Df size : 0.7
y_test shape: (393,) ,Test size / Df size : 0.3


Zbiór został podzielony poprawnie, widzimy, że stosunki poszczególnych zbiorów ze zbiorem oryginalnym pokrywają się z tym czego chcieliśmy.
(916,) brakująca wartość w shape, jest jednoznaczna z wartością "1", więc wszystko jest poprawnie

### 5. Zbadaj w zależności od kardynalności danej zmiennej czy rozkład etykiet dla poszczególnych cech w zbiorach testowych i treningowych jest równomierny.

In [24]:
for column in col_name:
    Unique_test = [x for x in X_train[column].unique() if x not in X_test[column].unique()]
    if len(Unique_test) == 0:
        print(f'Rozkład etykiety {column} jest równomierny')
    else:
        print(f'Rozkład etykiety {column} nie jest równomierny, różnica w ilości unikatowych etykiet wynosi {np.abs(len(X_test[column].unique()) - len(X_train[column].unique())) }')

Rozkład etykiety cabin nie jest równomierny, różnica w ilości unikatowych etykiet wynosi 80
Rozkład etykiety CabinReduced nie jest równomierny, różnica w ilości unikatowych etykiet wynosi 1
Rozkład etykiety sex jest równomierny


To, że rozkład etykiet w cabin i CabinReduced nie jest równomierny w obu zbiorach może powodować problemy. Mianowicie nasz model predykcyjny nie będzie wiedział co zrobić z tymi 80 etykietami, przy modelowaniu w oparciu o zbiór testowy

### 6. Wykonaj kodowanie zmiennych kategorycznych do zmiennych liczbowych. Wykorzystaj pętlę for i metodę enumerate(). Wynik powinien być podobny do tego na obrazku poniżej 

In [6]:
maps = {}
for column in col_name:
    column_map = {}
    for index, element in enumerate(df[column].unique()):
        column_map[element] = index
    maps[column + "_map"] = column_map
cabin_map, CabinReduced_map, sex_map = maps["cabin_map"], maps["CabinReduced_map"], maps["sex_map"]

### 7. Zastąp etykiety zmiennej (tu przykład dla kabina) słownikiem stworzonym w kroku 6. Do tego będzie potrzebne mapowanie.

In [7]:
for column in col_name:
    map_name = f'{column}_map'
    X_train[map_name] = X_train[column].map(maps[map_name])
    X_test[map_name] = X_test[column].map(maps[map_name])
display(X_train.head(10))
display(X_test.head(10))

,cabin,CabinReduced,sex,cabin_map,CabinReduced_map,sex_map
501,NaN,N,female,6,5,0
588,NaN,N,female,6,5,0
402,NaN,N,female,6,5,0
1193,NaN,N,male,6,5,1
686,NaN,N,female,6,5,0
971,NaN,N,male,6,5,1
117,E36,E,female,64,2,0
540,NaN,N,female,6,5,0
294,C68,C,male,157,1,1
261,E24,E,male,30,2,1


,cabin,CabinReduced,sex,cabin_map,CabinReduced_map,sex_map
1139,NaN,N,male,6,5,1
533,NaN,N,female,6,5,0
459,NaN,N,male,6,5,1
1150,NaN,N,male,6,5,1
393,NaN,N,male,6,5,1
1189,G6,G,female,185,8,0
5,E12,E,male,2,2,1
231,C104,C,male,130,1,1
330,NaN,N,male,6,5,1
887,NaN,N,male,6,5,1


### 8. Sprawdź liczbę brakujących wartości w zmodyfikowanych zbiorach. Zapisz wyniki i skomentuj.

In [8]:
train_missing_values = {}
for column in col_name:
    map_name = f'{column}_map'
    train_missing_values[map_name] = mapped_train = X_train[map_name].isna().sum() 
test_missing_values = {}
for column in col_name:
    map_name = f'{column}_map'
    test_missing_values[map_name] = mapped_train = X_test[map_name].isna().sum()

In [9]:
print("Zbiór treningowy:",train_missing_values)
print("Zbiór testowy:",test_missing_values)

Zbiór treningowy: {'cabin_map': 0, 'CabinReduced_map': 0, 'sex_map': 0}
Zbiór testowy: {'cabin_map': 0, 'CabinReduced_map': 0, 'sex_map': 0}


#### Analiza wyników
- Jak można się było spodziewać, w zmodyfikowanych zbiorach nie występują brakujące wartości, ponieważ zostały zmapowane na wartości liczbowe. Ta operacja w żaden sposób nie redukuje kardynalości cech
- Wydaje mi się, że jedyna korzyść wynikająca z mapowania wartości kategorycznych na wartości liczbowe całkowie, jest że, te wartości są łatwiejsze w interpretacji modelowi.
- Nie możemy nawet wykonać punktu 9, ponieważ nie ma wartości brakujących

### 9. Zastąp brakujące wartości liczbą 0. Czy jest to najlepsze wyjście?
- pominięte

### 10. Porównaj ile unikalnych wartości jest w zbiorze treningowym, a ile w zbiorze testowym (funkcja len). Jaka jest różnica pomiędzy liczbą etykiet przed i po redukcji oraz mapowaniu? 
### Czy cały proces, który został do tej pory wykonany może mieć wpływ na końcowy wynik predykcji i jakość modelu?

In [31]:
train_unique_len = 0
test_unique_len = 0
for column in col_name:
    map_name = f'{column}_map'
    train_unique_len += len(X_train[map_name].unique())
    test_unique_len += len(X_test[map_name].unique())
print(train_unique_len)
print(test_unique_len)

162
81


<div>Tak jak wykazaliśmy w punkcie 5, różnica pomiędzy ilością unikalnych etykiet się nie zmieniła i nadal wynosi 81, ponieważ mamy 80 etykiet w zmiennej cabin, które występują w zbiorze treningowym a w testowym już nie.</div>
<div>Oraz 1 etykietę w zmiennej CabinReduced, która występuje w zbiorze treningowym a w testowym już nie.</div>
<div><br></div>
<div>Cały proces wykonany w tym notatniku, tj. podział na zbiór testowy i trenigowy oraz mapowanie zmiennych kategorycznych na liczby całkowite.</div>
<div>Podział zbioru danych na 4 pomniejsze, jest niezbędny do tworzenia modelu i jego testowania, ponieważ, aby model dział poprawnie, nie możemy go trenować na całości danych (unikamy overfittingu) jednocześnie testując go, na trochę 'innym' zbiorze, by sprawdzić poprawność działana</div>
<div>więc ten krok zdecydowanie wpływa na wynik predykcji i jakość modelu w pozytywny sposób</div>
<div>Druga część, czyli mapowanie zmiennych, w postaci jakiej to zrobiliśmy nie ma dla mnie większego sensu, szczególnie zostawienie kolumn zmapowanych oraz oryginalnych w zbiorach treningowym oraz testowym jest kontr-produktywne, ponieważ są to basically te same dane</div>
<div>Logicznym postępowaniem byłoby pozbycie się oryginalnych kolumn, ponieważ te zmapowane będą prostsze w interpretacji dla naszego modelu predykcyjnego</div>
<div>Koniec końców, operacja mapowania na tym etapie nie wiele wnosi, i można właściwie ją wykonać pod koniec, czyli przed 'wrzucaniem' danych do modelu, ponieważ istnieje szansa, że naprzykład będziemy chcieli się pozbyć brakujących danych kompletnie, a mapowanie nam to utrudni</div>